In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
from sklearn.decomposition import PCA
import cv2
import os 
from skimage import filters
from scipy import ndimage

In [2]:
from glob import glob

In [3]:
%run auxfunctions.ipynb

In [5]:

import glob

path = '../ck+copyAgainT/Landmarks/'

files = [f for f in glob.glob(path + "**/*.txt", recursive=True)]
files
frames_lmarks_XYord = get_landmarks_of_folder(files)


In [6]:
frames_lmarks_XYord.shape

(4064, 136)

In [10]:
 np.sort(glob("../ck+copyAgainT/Landmarks/*/*/", recursive = True))

array(['../ck+copyAgainT/Landmarks/S010/002/',
       '../ck+copyAgainT/Landmarks/S010/004/',
       '../ck+copyAgainT/Landmarks/S010/006/',
       '../ck+copyAgainT/Landmarks/S011/001/',
       '../ck+copyAgainT/Landmarks/S011/002/',
       '../ck+copyAgainT/Landmarks/S011/003/',
       '../ck+copyAgainT/Landmarks/S011/004/',
       '../ck+copyAgainT/Landmarks/S011/005/',
       '../ck+copyAgainT/Landmarks/S011/006/',
       '../ck+copyAgainT/Landmarks/S014/001/',
       '../ck+copyAgainT/Landmarks/S014/002/',
       '../ck+copyAgainT/Landmarks/S014/003/',
       '../ck+copyAgainT/Landmarks/S014/005/',
       '../ck+copyAgainT/Landmarks/S022/001/',
       '../ck+copyAgainT/Landmarks/S022/005/',
       '../ck+copyAgainT/Landmarks/S022/006/',
       '../ck+copyAgainT/Landmarks/S026/001/',
       '../ck+copyAgainT/Landmarks/S026/002/',
       '../ck+copyAgainT/Landmarks/S026/003/',
       '../ck+copyAgainT/Landmarks/S026/006/',
       '../ck+copyAgainT/Landmarks/S029/001/',
       '../ck

In [14]:
#Vamos a sacar la primera y última frame de cada serie, asi como las neutrales que sobran..
#.(no, las neutrales que sobran combiene sacarlas al final.)
#Si es de fear o sadness, sacamos las dos últimas
from glob import glob

path_of_used_series_landmarks = []
path_of_used_series_images = []
for i in np.sort(glob("../ck+copyAgainT/Landmarks/*/*/", recursive = True)):
    subject = i.split('/')[3]
    serie = i.split('/')[4]
    actual = subject + '/'+ serie
    path = '../ck+copyAgainT/Emotion_labels/Emotion/' +actual + '/'
    image_path = '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/' +actual+ '/'
    landmark_path = '../ck+copyAgainT/Landmarks/' + actual+ '/'
   
    
    longitud = len(np.sort((os.listdir(str(i)))))   
    imagelabel = np.sort(os.listdir(str(image_path))) 
    landmarklabel = np.sort(os.listdir(str(landmark_path)))
    
    filelabel = (os.listdir(path))[0]
    with open(path+ '/'+filelabel) as f:        
        line = float(f.readline())        
    if line == 4 or line == 6:
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[0])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[(longitud)-4])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[(longitud)-3]) 
        
        path_of_used_series_images.append(image_path + imagelabel[0])
        path_of_used_series_images.append(image_path + imagelabel[(longitud)-4])
        path_of_used_series_images.append(image_path + imagelabel[(longitud)-3])       
    else:
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[0])
        path_of_used_series_landmarks.append(landmark_path + landmarklabel[(longitud)-2])       
        
        path_of_used_series_images.append(image_path + imagelabel[0])
        path_of_used_series_images.append(image_path + imagelabel[(longitud)-2])    
        
path_of_used_series_images

#vamos a hacer el pca con las neutrales y sin las neutrales.
path_of_used_series_landmarks


['../ck+copyAgainT/Landmarks/S010/002/S010_002_00000001_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S010/002/S010_002_00000013_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S010/004/S010_004_00000001_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S010/004/S010_004_00000018_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S010/006/S010_006_00000001_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S010/006/S010_006_00000014_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/001/S011_001_00000001_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/001/S011_001_00000015_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/002/S011_002_00000001_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/002/S011_002_00000019_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/002/S011_002_00000020_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/003/S011_003_00000001_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/003/S011_003_00000011_landmarks.txt',
 '../ck+copyAgainT/Landmarks/S011/003/S011_003_00000012_landmark

In [67]:
path_of_used_series_images

['../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S010/002/S010_002_00000001.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S010/002/S010_002_00000013.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S010/004/S010_004_00000001.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S010/004/S010_004_00000018.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S010/006/S010_006_00000001.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S010/006/S010_006_00000014.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S011/001/S011_001_00000001.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S011/001/S011_001_00000015.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S011/002/S011_002_00000001.png',
 '../ck+copyAgainT/extended-cohn-kanade-images/cohn-kanade-images/S011/002/S011_002_00000019.png',
 '../ck+co

In [16]:
frames_lmarks_XYord = get_landmarks_of_folder(path_of_used_series_landmarks)

In [17]:
images = path_of_used_series_images
images = np.sort(np.asarray(images))

'S010_006_00000014.png'

In [18]:
intercalated_data = intercalate_data_of_vectors(frames_lmarks_XYord, 0)


In [34]:
import matplotlib.cm as cm 
my_landmarks = intercalated_data
pathi = '../ck+warpedAgainT_V2'
transformed_landmarks_total = align_and_transform_vector_of_images(images[0], my_landmarks[0], my_landmarks, images, pathi)

In [22]:
transformed_landmarks_total.shape

(480, 68, 2)

In [37]:
transformed_final = transformed_landmarks_total[0].flatten('F')
for i in range (1, len(transformed_landmarks_total)):
    transformed_final = np.vstack((transformed_final, transformed_landmarks_total[i].flatten('F')))
transformed_final[0]


my_df = pd.DataFrame(transformed_final)
my_df.to_csv('transformed_landmarks_procustres_AgainTV2_totalxy.csv',header = False, index= False)

In [9]:
my_data = pd.read_csv('transformed_landmarks_procustres_AgainTV2_totalxy.csv',header = None)
ind = 0
my_data_aligned_diff = []
images_names = np.sort(os.listdir('../ck+warpedAgainT_V2'))
current_neutral_index = 0
for i in range(0, len(images_names)):
    if int(images_names[i].split('_')[5].split('.')[0]) == 1:
        current_neutral_index = i
        print('cambie neutral '+ str(current_neutral_index))
    
    my_data_aligned_diff.append(my_data.iloc[i]- my_data.iloc[current_neutral_index])
    
        
my_data_aligned_diff = np.asarray(my_data_aligned_diff)    


FileNotFoundError: [Errno 2] No such file or directory: 'transformed_landmarks_procustres_AgainTV2_totalxy.csv'

In [40]:
my_data_aligned_diff.shape

(480, 136)

In [41]:
my_data_aligned_diff = my_data_aligned_diff[~np.all(my_data_aligned_diff == 0, axis=1)]

In [43]:
my_data_aligned_diff.shape

(259, 136)

In [46]:
my_df = pd.DataFrame(my_data_aligned_diff)

my_df.to_csv('my_training_data_aligned_diff_AgainT_V2.csv',header = False, index= False)

In [4]:
my_data_forPCA1 = pd.read_csv('my_data_for_PCA_SNeu_AgainT_V1.csv',header = None)
#my_training_data_aligned_diff_AgainT_V1 <- esto para hacelo con neutrales(480)
my_data_forPCA2 = pd.read_csv('my_training_data_aligned_diff_AgainT_V2.csv',header = None)


In [52]:
331+259

590

In [5]:

my_data_forPCA = pd.concat([my_data_forPCA1, my_data_forPCA2])

In [6]:

len(my_data_forPCA)

739

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [11]:
def doPCA(data, comp):
    std_scaler = StandardScaler()
    scaler = std_scaler.fit(data)
    scaled_landDiff = scaler.transform(data)
    pca = PCA(n_components=comp)
    pac = pca.fit(scaled_landDiff)
    b_shape = pac.transform(scaled_landDiff)
    print(sum(pac.explained_variance_ratio_))
    print(b_shape.shape)
    return b_shape, pac, scaler

In [12]:

shape, pac, scaler = doPCA(my_data_forPCA, 29)

0.9624989781129226
(739, 29)


In [13]:
import pickle as pk
with open('pca_shapeWNeuV2.pkl', 'wb') as pickle_file:
        pk.dump(pac, pickle_file)
with open('scaler_shapeWNeuV2.pkl', 'wb') as pickle_file:
        pk.dump(scaler, pickle_file)

In [15]:


with open('pca_shapeWNeuV2.pkl', 'rb') as pickle_file:
    pca = pk.load(pickle_file)
with open('scaler_shapeWNeuV2.pkl', 'rb') as pickle_file:
    sl = pk.load(pickle_file)
pca


PCA(n_components=29)

In [13]:
my_data_forPCA1 = pd.read_csv('my_data_for_PCA_SNeu_AgainT_V1.csv',header = None)
my_data_forPCA2 = pd.read_csv('my_training_data_aligned_diff_AgainT_V2.csv',header = None)
my_data_forPCA3 = pd.read_csv('my_training_data_aligned_diff_AgainT_V3.csv',header = None)
my_data_forPCA = pd.concat([my_data_forPCA1, my_data_forPCA2, my_data_forPCA3])
len(my_data_forPCA)
shape, pac, scaler = doPCA(my_data_forPCA, 29)
import pickle as pk
with open('pca_shapeV3.pkl', 'wb') as pickle_file:
        pk.dump(pac, pickle_file)
with open('scaler_shapeV3.pkl', 'wb') as pickle_file:
        pk.dump(scaler, pickle_file)
with open('pca_shapeV3.pkl', 'rb') as pickle_file:
    pca = pk.load(pickle_file)
with open('scaler_shapeV3.pkl', 'rb') as pickle_file:
    sl = pk.load(pickle_file)
pca

0.9612503673334454
(849, 29)


PCA(n_components=29)

In [14]:
sl

StandardScaler()

In [16]:
my_df = pd.DataFrame(shape)
#my_df.to_csv('pca_training_shapepoints_AgainT_WNeuV2.csv',header = False, index= False)
my_df.to_csv('pca_training_shapepoints_AgainT_V2_SNeu.csv',header = False, index= False)

### ahora vamos a structure

In [64]:
my_data = pd.read_csv('transformed_landmarks_procustres_AgainTV2_totalxy.csv',header = None)
my_data #orden xy

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,265.050800,265.134280,266.581590,273.107110,285.159460,305.647910,326.266310,351.728120,383.572310,416.076590,...,403.721050,384.339700,365.114420,352.872000,351.195620,351.417280,365.428210,383.166430,393.743530,381.331230
1,268.372648,268.523696,270.196174,277.030778,289.745604,307.484740,327.475426,352.465115,383.631481,415.479025,...,410.033599,388.542133,365.812881,350.148823,346.495185,348.427939,366.548944,388.553865,401.089656,386.828793
2,257.326168,258.221938,260.805877,266.690325,276.474686,294.626252,320.389932,350.903551,384.016598,416.652835,...,381.291440,369.144658,363.629485,362.510836,363.795650,361.240308,361.083938,362.222985,365.226899,363.423345
3,253.456447,254.056917,256.595184,264.003770,274.286312,293.591422,321.147208,352.061710,386.971920,420.654689,...,365.399839,360.678656,357.688968,356.905326,359.921613,356.462408,356.413184,357.009660,360.221276,357.729584
4,254.903363,256.296142,258.279922,265.854494,273.228336,293.755002,321.060229,352.685877,387.297974,420.440242,...,380.992998,369.192722,362.104119,364.336818,365.953482,362.861085,361.016295,363.270506,366.821005,364.637481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,246.472393,248.380416,254.059928,260.002762,268.385993,286.878623,313.995946,347.537238,383.367737,419.716659,...,374.724052,372.139460,355.797698,355.228702,353.839377,353.498967,353.093587,353.751973,353.963433,355.355260
476,241.952888,244.041695,250.030614,256.313249,265.308487,285.131864,312.348143,346.123085,383.092443,420.952687,...,362.005165,358.145066,352.119371,351.558434,350.143439,349.907283,349.476450,350.134387,350.270435,351.721784
477,246.884274,245.567993,250.834303,256.640029,264.260362,283.502885,312.177996,347.341648,383.992157,420.255053,...,371.250844,368.623758,354.486332,353.861859,352.152094,352.137879,351.761961,352.348319,352.192229,354.161028
478,250.046423,248.576378,254.217372,261.038590,267.039701,284.226760,312.674930,347.297675,383.660595,420.258539,...,383.497150,378.421805,359.335595,346.467453,346.615505,347.267787,356.450847,366.834351,370.001805,371.099828


In [73]:
subjects_images = '../ck+warpedAgainT_V2'

In [80]:
first_frames = get_first_frames(subjects_images)
first_images= get_first_images(subjects_images)


In [82]:
intercalated_first_frames = change_to_intercalate_order(first_frames)
first_frame = intercalated_first_frames

In [83]:
import math
first_feature_point = list_get_feature_point(first_frame, 1, first_images)
second_feature_point = list_get_feature_point(first_frame, 2, first_images)
third_feature_point = list_get_feature_point(first_frame, 3, first_images)
fourth_feature_point = list_get_feature_point(first_frame, 4, first_images)
five_feature_point = list_get_feature_point(first_frame, 5, first_images)
six_feature_point = list_get_feature_point(first_frame, 6, first_images)
seven_feature_point = list_get_feature_point(first_frame, 7, first_images)
eight_feature_point = list_get_feature_point(first_frame, 8, first_images)
nine_feature_point = list_get_feature_point(first_frame, 9, first_images)
ten_feature_point = list_get_feature_point(first_frame, 10, first_images)
eleven_feature_point = list_get_feature_point(first_frame, 11, first_images)
twelve_feature_point = list_get_feature_point(first_frame, 12, first_images)
thirteen_feature_point = list_get_feature_point(first_frame, 13, first_images)
fourteen_feature_point = list_get_feature_point(first_frame, 14, first_images)
fifteen_feature_point = list_get_feature_point(first_frame, 15, first_images)
sixteen_feature_point = list_get_feature_point(first_frame, 16, first_images)
seventeen_feature_point = list_get_feature_point(first_frame, 17, first_images)
eigtheen_feature_point = list_get_feature_point(first_frame, 18, first_images)
nineteen_feature = list_get_feature_point(first_frame, 19, first_images)
twenty_feature = list_get_feature_point(first_frame, 20, first_images)
twentyone_feature = list_get_feature_point(first_frame, 21, first_images)

In [84]:

structure_vector = first_feature_point
structure_vector = np.vstack((structure_vector, second_feature_point))
structure_vector = np.vstack((structure_vector, third_feature_point))
structure_vector = np.vstack((structure_vector, fourth_feature_point))
structure_vector = np.vstack((structure_vector, five_feature_point))
structure_vector = np.vstack((structure_vector, six_feature_point))
structure_vector = np.vstack((structure_vector, seven_feature_point))
structure_vector = np.vstack((structure_vector, eight_feature_point))
structure_vector = np.vstack((structure_vector, nine_feature_point))
structure_vector = np.vstack((structure_vector, ten_feature_point))
structure_vector = np.vstack((structure_vector, eleven_feature_point))
structure_vector = np.vstack((structure_vector, twelve_feature_point))
structure_vector = np.vstack((structure_vector, thirteen_feature_point))
structure_vector = np.vstack((structure_vector, fourteen_feature_point))
structure_vector = np.vstack((structure_vector, fifteen_feature_point))
structure_vector = np.vstack((structure_vector, sixteen_feature_point))
structure_vector = np.vstack((structure_vector, seventeen_feature_point))
structure_vector = np.vstack((structure_vector, eigtheen_feature_point))
structure_vector = np.vstack((structure_vector, nineteen_feature))
structure_vector = np.vstack((structure_vector, twenty_feature))
structure_vector = np.vstack((structure_vector, twentyone_feature))


In [85]:
structure_vector.shape 

(21, 221)

In [96]:
len(my_data_21)

480

In [87]:
my_data_21 = pd.read_csv('my_training_data_aligned_diff_AgainT_V1.csv',header = None)
my_data_22 = pd.read_csv('my_training_data_aligned_diff_AgainT_V2.csv',header = None)
my_data_2 =  pd.concat([my_data_21, my_data_22])
my_data_2

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.321848,3.389416,3.614584,3.923668,4.586144,1.836830,1.209116,0.736995,0.059171,-0.597565,...,6.312549,4.202433,0.698461,-2.723177,-4.700435,-2.989341,1.120734,5.387435,7.346126,5.497563
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-3.865982,-4.154327,-4.198715,-2.676183,-2.193575,-1.075355,0.691931,1.080560,2.856966,3.918157,...,-15.640824,-8.321081,-5.836411,-5.513324,-3.785196,-4.675919,-4.567765,-5.097905,-4.901998,-5.592174
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,1.862863,3.268257,3.465489,3.907600,4.620467,2.678868,1.624597,1.108570,0.215799,-1.001761,...,-1.014054,2.200598,1.746667,1.573042,0.451594,0.760686,0.868409,1.178258,0.513963,1.732883
255,1.876813,3.314217,3.520227,3.951406,4.672227,2.723416,1.648907,1.112803,0.208599,-1.023097,...,-1.173247,2.157114,1.696605,1.523522,0.371497,0.655288,0.759772,1.070563,0.414347,1.679523
256,-4.519505,-4.338721,-4.029314,-3.689513,-3.077506,-1.746759,-1.647803,-1.414153,-0.275294,1.236027,...,-12.718887,-13.994394,-3.678327,-3.670268,-3.695937,-3.591684,-3.617137,-3.617586,-3.692998,-3.633476
257,3.162148,3.008386,3.383069,4.398561,2.779340,0.723875,0.496934,-0.043972,-0.331562,0.003486,...,12.246306,9.798047,4.849263,-7.394406,-5.536589,-4.870093,4.688886,14.486032,17.809577,16.938800


In [88]:
Ai = my_data_2.max() # da el maximo por columnas
Ai.shape

(136,)

In [138]:
data_2_intensity = my_data_2
data_2_intensity_div = data_2_intensity/Ai
data_2_intensity_sum =  np.sum(data_2_intensity_div, axis=1)
data_2_intensity_final = data_2_intensity_sum/136
data_2_intensity_final  #the emotion for each frame
#revisar que este bien la cuenta

0      0.000000
1     -0.053311
2      0.000000
3      0.056637
4      0.000000
         ...   
254   -0.043226
255   -0.044049
256    0.029559
257   -0.140057
258   -0.140719
Length: 739, dtype: float64

In [139]:
data_2_intensity_final.shape

(739,)

In [100]:
images_names[10].split('_')[3]


'S011'

In [127]:
images_names = np.sort(os.listdir('../ck+warpedAgainT_V2'))    
#images_names

In [142]:
data_2_intensity_final.iloc[0]

0.0

In [145]:
cant_features = 22
current_sub = 'start'
#frame_seq_flattened = frame_seq_array.flatten()
ab = 0
structure_feature_vector = np.zeros((len(my_data_22), cant_features))

#total = 0
    
current_features_index = -1
#images_names = np.sort(os.listdir('../ck+warpedAgainT_V1'))   
no_neutral_im
for i in range(480, len(data_2_intensity_final)):        
    if  no_neutral_im[ab].split('_')[3] != current_sub:
        current_sub = no_neutral_im[ab].split('_')[3] 
        #print(i, current_features_index )
        current_features_index = current_features_index + 1
    vector = structure_vector[:,current_features_index]#columna i
    vector = np.append(vector,data_2_intensity_final.iloc[i]) 
    structure_feature_vector[ab] = vector
    ab = ab + 1
    
        



480 -1
483 0
491 1
496 2
499 3
504 4
505 5
511 6
514 7
517 8
520 9
523 10
528 11
530 12
533 13
535 14
539 15
545 16
548 17
551 18
554 19
557 20
560 21
562 22
566 23
571 24
576 25
580 26
585 27
588 28
590 29
595 30
599 31
601 32
604 33
608 34
612 35
615 36
619 37
621 38
624 39
627 40
630 41
635 42
638 43
641 44
644 45
648 46
649 47
653 48
657 49
661 50
664 51
665 52
666 53
670 54
674 55
678 56
682 57
683 58
688 59
695 60
700 61
703 62
709 63
714 64
717 65
722 66
726 67
729 68
731 69
736 70


(259, 22)

'S010'

In [118]:
images_names = np.sort(os.listdir('../ck+warpedAgainT_V2'))    
len(images_names)

480

In [124]:
no_neutral_im = []
for i in range (0, len(images_names)):
    if int(images_names[i].split('_')[5].split('.')[0]) != 1:
        no_neutral_im.append(images_names[i])
        
no_neutral_im = np.asarray(no_neutral_im)
no_neutral_im.shape

(259,)

In [148]:
structure_feature_vector = np.asarray(structure_feature_vector)
my_df = pd.DataFrame(structure_feature_vector)
my_df.to_csv('structure_feature_vector_AgainT_V2.csv',header = False, index= False)

In [19]:
#my_data_forPCA1 = pd.read_csv('my_data_for_PCA_structure_SNeu_AgainT_V1.csv',header = None)
my_data_forPCA1 = pd.read_csv('structure_feature_vector_AgainT_V1.csv',header = None)#<-con NEuro
my_data_forPCA2 = pd.read_csv('structure_feature_vector_AgainT_V2.csv',header = None)
my_data_for_PCAss = pd.concat([my_data_forPCA1, my_data_forPCA2])

In [20]:

len(my_data_for_PCAss)

739

In [21]:

struct, pac, scaler = doPCA(my_data_for_PCAss, 13)

0.9441897424296163
(739, 13)


In [22]:
with open('scaler_structureWNeuV2.pkl', 'wb') as pickle_file:
        pk.dump(scaler, pickle_file)
with open('scaler_structureWNeuV2.pkl', 'rb') as pickle_file:
    sl = pk.load(pickle_file)
with open('pca_structureWNeuV2.pkl', 'wb') as pickle_file:
        pk.dump(pac, pickle_file)
with open('pca_structureWNeuV2.pkl', 'rb') as pickle_file:
    pca = pk.load(pickle_file)
sl

StandardScaler()

In [22]:
my_data_forPCA1 = pd.read_csv('my_data_for_PCA_structure_SNeu_AgainT_V1.csv',header = None)
my_data_forPCA2 = pd.read_csv('structure_feature_vector_AgainT_V2.csv',header = None)
my_data_forPCA3 = pd.read_csv('structure_feature_vector_AgainT_V3.csv',header = None)
my_data_for_PCAss = pd.concat([my_data_forPCA1, my_data_forPCA2, my_data_forPCA3])
len(my_data_for_PCAss)
struct, pac, scaler = doPCA(my_data_for_PCAss, 13)
with open('scaler_structureV3.pkl', 'wb') as pickle_file:
        pk.dump(scaler, pickle_file)
with open('scaler_structureV3.pkl', 'rb') as pickle_file:
    sl = pk.load(pickle_file)
with open('pca_structureV3.pkl', 'wb') as pickle_file:
        pk.dump(pac, pickle_file)
with open('pca_structureV3.pkl', 'rb') as pickle_file:
    pca = pk.load(pickle_file)
sl

0.950496728310322
(849, 13)


StandardScaler()

In [23]:
my_df = pd.DataFrame(struct)
my_df.to_csv('pca_training_structurepoints_AgainT_WNeuV2.csv',header = False, index= False)
#my_df.to_csv('pca_training_structurepoints_AgainT_V2_SNeu.csv',header = False, index= False)

In [159]:
subject_images = []
warpimgpath = '../ck+warpedAgainT_V2/'
os.listdir(warpimgpath)
for i in np.sort(os.listdir(warpimgpath)):  
    subject_images.append(warpimgpath + i)
subject_images = np.asarray(subject_images)

img = cv2.imread(subject_images[0])
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = img[40:450, 170:580]
cv2.imshow("Texture start", img)
# Pause screen to wait key from user to see result
cv2.waitKey(0)
cv2.destroyAllWindows()


In [161]:
subject_images.shape

(480,)

In [168]:
no_neutral_im2 = []
for i in range (0, len(subject_images)):
    if int(subject_images[i].split('_')[6].split('.')[0]) != 1:
        no_neutral_im2.append(subject_images[i])
        
no_neutral_im2 = np.asarray(no_neutral_im2)
no_neutral_im2.shape
subject_images = no_neutral_im2

In [171]:
len(subject_images)

259

In [172]:
for i in range (0,  len(subject_images)):#last time we stopped last finished  in 688
    
    subject = subject_images[i].split('/')[2].split('_')[3]
    serie = subject_images[i].split('/')[2].split('_')[4]
    frame = subject_images[i].split('/')[2].split('_')[5].split('.')[0]
    print(subject,serie, frame)
    img = cv2.imread(subject_images[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img2 = img[40:450, 170:580]
    print(img2.shape)
    print(i)
    cartoon, texture = cartoonTexture_grey(img2, 0.2, 50, 0.35, 0.35, 1.0, 0)
    cv2.imwrite("cartoon_AgainT_V2/cartoonimg"+'_' + subject +'_' +serie +'_'+frame +".png", cartoon)
    cv2.imwrite("texture_AgainT_V2/textureimg"+'_' + subject +'_' +serie +'_'+frame +".png", texture)

S010 002 00000013
(410, 410)
0
S010 004 00000018
(410, 410)
1
S010 006 00000014
(410, 410)
2
S011 001 00000015
(410, 410)
3
S011 002 00000019
(410, 410)
4
S011 002 00000020
(410, 410)
5
S011 003 00000011
(410, 410)
6
S011 003 00000012
(410, 410)
7
S011 004 00000020
(410, 410)
8
S011 005 00000019
(410, 410)
9
S011 006 00000012
(410, 410)
10
S014 001 00000028
(410, 410)
11
S014 002 00000013
(410, 410)
12
S014 002 00000014
(410, 410)
13
S014 003 00000029
(410, 410)
14
S014 005 00000016
(410, 410)
15
S022 001 00000029
(410, 410)
16
S022 005 00000031
(410, 410)
17
S022 006 00000016
(410, 410)
18
S026 001 00000014
(410, 410)
19
S026 002 00000013
(410, 410)
20
S026 002 00000014
(410, 410)
21
S026 003 00000014
(410, 410)
22
S026 006 00000012
(410, 410)
23
S029 001 00000018
(410, 410)
24
S032 001 00000021
(410, 410)
25
S032 003 00000016
(410, 410)
26
S032 004 00000011
(410, 410)
27
S032 004 00000012
(410, 410)
28
S032 005 00000015
(410, 410)
29
S032 006 00000015
(410, 410)
30
S034 001 00000028


S506 004 00000035
(410, 410)
252
S506 004 00000036
(410, 410)
253
S506 006 00000039
(410, 410)
254
S506 006 00000040
(410, 410)
255
S999 001 00000017
(410, 410)
256
S999 003 00000052
(410, 410)
257
S999 003 00000053
(410, 410)
258


In [175]:
path = 'cartoon_AgainT_V2/'
cartoon_images = []
for i in np.sort(os.listdir(path)):
  cartoon_images.append( path + i)

cartoon_images

imcart = []
for i in cartoon_images:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    img_mean = img/np.mean(img)
    imcart.append(img_mean.flatten())

In [178]:
path = 'cartoon_AgainT_V1/'
cartoon_imagesn = []
for i in np.sort(os.listdir(path)):
  cartoon_imagesn.append( path + i)

cartoon_imagesn

imcartn = []
for i in cartoon_imagesn:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    img_mean = img/np.mean(img)
    imcartn.append(img_mean.flatten())

In [187]:
np.sort(os.listdir('../ck+warpedAgainT_V1'))[0].split('_')[5].split('.')[0]

'00000001'

In [196]:
no_neutral_im3 = []
j = 0
for i in  np.sort(os.listdir('../ck+warpedAgainT_V1')):
    if int(i.split('_')[5].split('.')[0]) == 1:
        no_neutral_im3.append(imcartn[j])
    j = j+1
        
no_neutral_im3 = np.asarray(no_neutral_im3)
no_neutral_im3.shape

(221, 168100)

In [203]:
len(cartoon_images)

259

In [204]:
imcartoon = []
index = -1
subject = 'start'
current_image = imcart[0]
for i in range(0, len(cartoon_images)):
    if ((cartoon_images[0].split('/')[1].split('_')[1]) != subject):
        subject = (cartoon_images[0].split('/')[1].split('_')[1])
        index = i
        current_image = no_neutral_im3[index]
    imcartoon.append(imcart[i]- current_image)


In [206]:
len(imcartoon)

259

In [207]:
imcartoon = np.asarray(imcartoon)
imcartoon.shape

(259, 168100)

In [208]:
np.savetxt('im_AgainT_V2_cartoonarray_.csv', imcartoon, delimiter=",")


In [209]:
texture_images = []
path = 'texture_AgainT_V2/'
for i in np.sort(os.listdir(path)):
    texture_images.append( path + i)

In [210]:
imtext = []
for i in texture_images:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    imtext.append(img.flatten())

imtextarray = np.asarray(imtext)
imtextarray.shape

(259, 168100)

In [211]:
np.savetxt('im_AgainT_V2_texturearray_.csv', imtextarray, delimiter=",")

In [32]:
file_path = 'im_AgainT_V1_cartoonarray_.csv'
chunk_size = 100

df_list = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size, header = None):
    df_list.append(chunk)
    
file_path = 'im_AgainT_V2_cartoonarray_.csv'

for chunk in pd.read_csv(file_path, chunksize=chunk_size, header = None):
    df_list.append(chunk)

my_data_for_PCAsc = pd.concat(df_list)

FileNotFoundError: [Errno 2] No such file or directory: 'im_AgainT_V1_cartoonarray_.csv'

In [9]:
cartoon, pac = doPCA(my_data_for_PCAsc, 100)

0.9533231940692543
(739, 100)


In [11]:
my_df = pd.DataFrame(cartoon)
my_df.to_csv('pca_cartoonAgainT_V2_points_.csv',header = False, index= False)

In [6]:
file_path = 'im_AgainT_V1_texturearray_.csv'
chunk_size = 100

df_list = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size, header = None):
    df_list.append(chunk)
    
file_path = 'im_AgainT_V2_texturearray_.csv'

for chunk in pd.read_csv(file_path, chunksize=chunk_size, header = None):
    df_list.append(chunk)

my_data_for_PCAst = pd.concat(df_list)

In [ ]:
texture = doPCA(my_data_for_PCAst, 250)

In [ ]:
my_df = pd.DataFrame(texture)
my_df.to_csv('SynthesisEmotions/Notebook/pca_textureAgainT_V2_points_.csv',header = False, index= False)

In [7]:
cartoon_images[i].split('/')[1]

'cartoonimg_S010_002_00000001.png'

In [5]:
path = 'cartoon_AgainT_V1/'
cartoon_images = []
for i in np.sort(os.listdir(path)):
  cartoon_images.append( path + i)

cartoon_images

imcart = []
for i in cartoon_images:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    img_mean = img/np.mean(img)
    imcart.append(img_mean.flatten())


In [8]:
imcartoon = []
index = -1
subject = 'start'
current_image = imcart[0]
for i in range(0, len(cartoon_images)):
  if (int(cartoon_images[i].split('/')[1].split('_')[3].split('.')[0]) == 1):
    index = i
    current_image = imcart[i]
  imcartoon.append(imcart[i]- current_image)

imcartoon = np.asarray(imcartoon)
imcartoon.shape

(480, 168100)

In [9]:
np.savetxt('im_AgainT_V1_cartoonarray_.csv', imcartoon, delimiter=",")


In [10]:
texture_images = []
path = 'texture_AgainT_V1/'
for i in np.sort(os.listdir(path)):
  texture_images.append( path + i)

imtext = []
for i in texture_images:
    img = cv2.imread(i, cv2.IMREAD_GRAYSCALE)
    imtext.append(img.flatten())

imtextarray = np.asarray(imtext)
imtextarray.shape


(480, 168100)

In [11]:
np.savetxt('im_AgainT_V1_texturearray_.csv', imtextarray, delimiter=",")